# 머신러닝 with Python 
## 09 앙상블 학습 배깅(Bootstrap aggregating) 연습- wine data set

##### 나이브 베이즈 모델로 해보았던 실험을 이어서 하기 위해 서포트 벡터 머신을 가져왔다.

물론 어떤 모델을 사용하는게 적합한지는 데이터에 따라 다르고, 와인 데이터 실습에는 나이브 베이즈가 적합한 이유가 있겠지만 

지금 알고 싶은 것은 분류기의 개수에 따른 정확도 실험 결과가 다른 분류기에서도 같은 결과인지를 알고 싶다.

어떤 데이터로 실험하느냐에 따라 다르겠지만, 의미없다해도 눈으로 확인하고싶다.

책 p.267~ (실습은 Gaussian Naive Bayes 모델로 진행되었지만 Support Vector Machine으로 해보기로 한다.)

*진행 과정 중 코드에서 막힌 부분은 https://m.blog.naver.com/cjh226/221359032956 을 참고하였다*

---

하단부에서는 gnb와 마찬가지로 분류기의 개수에 따른 성능의 변화를 실험해 보았다.

배운점은 gnb와 비교해보면서 적었다.

> 배운점1: svm은 개수에 큰 영향없이 처음부터 성능이 거의 일정하다.

> 배운점2: 분류기의 개수가 늘어난다고 하여 꼭 성능이 저하되거나 하진 않는다.

> 배운점3: 분류 결과를 종합하는 방법이 같다면 분류기의 개수가 일정 수준이상 늘어나더라도 성능은 변하지 않는 선이 있다.


### 데이터 불러오기

In [1]:
from sklearn import datasets

In [2]:
raw_wine = datasets.load_wine() # 와인 데이터 가져오기

In [3]:
# 데이터 살펴보기
raw_wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [4]:
raw_wine.feature_names      # 데이터 셋 내 피처 이름들

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

### 피처, 타깃 데이터 지정

In [5]:
X = raw_wine.data
y = raw_wine.target

### 트레이닝/테스트 데이터 분할

In [6]:
from sklearn.model_selection import train_test_split                 # 분할을 위해 필요한 함수
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state = 0)    # 분리. randomstate는 고정

### 데이터 표준화

In [7]:
from sklearn.preprocessing import StandardScaler         # 데이터 표준화를 위한 함수
std_scale = StandardScaler()                             # 표준화 스케일러 지정
std_scale.fit(X_tn)                                      # 트레이닝 피처를 기준으로 표준화를 적합 시도

X_tn_std = std_scale.transform(X_tn) 
X_te_std = std_scale.transform(X_te)                     # 트레인, 테스트 데이터 각각 적합시킨 표준화에 맞게 변형

### 데이터 학습

In [12]:
from sklearn.svm import SVC, LinearSVC                   # 서포트 벡터 머신 분류기
from sklearn.ensemble import BaggingClassifier           # 배깅 앙상블을 사용하여 분류

In [16]:
# 배깅을 활용한 분류기 생성
clf_bagging_svc = BaggingClassifier(base_estimator = SVC(),         # base_estimator : 개별 학습기 입력. 선택하지 않으면 의사결정트리가 설정된다.
                                    n_estimators = 10,              # n_estimator : 개별 학습기의 개수. Default = 10
                                    random_state = 0)               # 고정

clf_bagging_svc.fit(X_tn_std, y_tn)                                 # 적합시키기

BaggingClassifier(base_estimator=SVC(), random_state=0)

### 데이터 예측

In [17]:
pred_bagging_svc = clf_bagging_svc.predict(X_te_std)                # std된 테스트 피처 데이터를 넣고 실행하여 결과 확인 
print(pred_bagging_svc)

[0 2 1 0 1 1 0 2 1 1 2 2 0 1 2 1 0 0 1 0 1 0 0 1 1 1 1 1 1 2 0 0 1 0 0 0 2
 1 1 2 0 0 1 1 1]


### 정확도 평가

In [18]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_te, pred_bagging_svc)                   # 실제값과 예측값을 넣음
print(accuracy)

1.0


### confusion matrix 확인

In [19]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_te, pred_bagging_svc)
print(conf_matrix)

[[16  0  0]
 [ 0 21  0]
 [ 0  0  8]]


### 분류 리포트 확인

In [20]:
from sklearn.metrics import classification_report
class_report = classification_report(y_te, pred_bagging_svc)
print(class_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



### 결과

1.0의 정확도가 나왔는데 언제나 그랬듯이 기분이 좋지않은 결과인 것이었다.

분류기 개수에 따라 성능을 체크해보기로 한다.

## 실험3: 분류기 개수 2~10개 각각 성능 체크해보기

In [21]:
from sklearn.svm import SVC                              # 서포트 벡터 머신 분류기
from sklearn.ensemble import BaggingClassifier           # 배깅 앙상블을 사용하여 분류
from sklearn.metrics import accuracy_score               # 정확도 테스트 함수
from sklearn.metrics import classification_report        # 분류 리포트 확인을 위한 함수


for i in range(2, 11):
    # 배깅을 활용한 분류기 생성
    clf_bagging_svc = BaggingClassifier(base_estimator = SVC(), 
                                        n_estimators = i,               # n_estimator : 개별 학습기의 개수
                                        random_state = 0)               # 고정
    
    # 데이터 학습
    clf_bagging_svc.fit(X_tn_std, y_tn)

    # 데이터 예측
    pred_bagging_svc = clf_bagging_svc.predict(X_te_std) 
    
    # 데이터 정확도 평가
    accuracy = accuracy_score(y_te, pred_bagging_svc)
    
    # 분류 리포트 확인
    class_report = classification_report(y_te, pred_bagging_svc)
    
    # 모델 상황 확인
    
    print("모델 분류기 개수: ", i )
    print("모델 정확도: ", accuracy)
    print("모델 분류 리포트")
    print(class_report)
    print("========================================================")

모델 분류기 개수:  2
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

모델 분류기 개수:  3
모델 정확도:  0.9777777777777777
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.95      0.98        21
           2       0.89      1.00      0.94         8

    accuracy                           0.98        45
   macro avg       0.96      0.98      0.97        45
weighted avg       0.98      0.98      0.98        45

모델 분류기 개수:  4
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
   

 ### 결과
 
 두둥 이게 무슨 일인가. 나이브 베이즈 분류기를 사용하였을 때와는 결과가 또 다르다.
 
 2개일 때 정확도가 1.0이지만, 3개에서 0.9777, 4개에서 1.0으로 올라갔다.
 
 여기서 든 의문! 혹시 3개에서 틀렸던 분류는 이전에 나이브 베이즈에서 1.0이 아닐때 꼭 틀렸던 클래스와 같을까?
 
 ---
 
 나이브 베이즈에서 정확도가 떨어지기 시작했던 분류기 개수가 4개였는데, 그때의 precision을 보면
 ```
 모델 분류기 개수:  4
모델 정확도:  0.9777777777777777
모델 분류 리포트
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.95      0.98        21
           2       1.00      1.00      1.00         8

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45
 ```
 
 여기서 정확도가 떨어졌더 분류기 개수 3개의 precision을 보면
 ```
 모델 분류기 개수:  3
모델 정확도:  0.9777777777777777
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      0.95      0.98        21
           2       0.89      1.00      0.94         8

    accuracy                           0.98        45
   macro avg       0.96      0.98      0.97        45
weighted avg       0.98      0.98      0.98        45
```

혹시나 특정 데이터가 컴퓨터에게 어려운(?) 데이터이진 않을까 살짝 기대도 했는데 이를 보니 아니었다.

---

결국 11개 이상의 결과는 뻔한 것 같지만 같은 실험은 진행해보아야 한다.

## 실험3: 분류기가 11개 이상일 떄

In [22]:
from sklearn.svm import SVC                              # 서포트 벡터 머신 분류기
from sklearn.ensemble import BaggingClassifier           # 배깅 앙상블을 사용하여 분류
from sklearn.metrics import accuracy_score               # 정확도 테스트 함수
from sklearn.metrics import classification_report        # 분류 리포트 확인을 위한 함수


for i in range(11, 21):
    # 배깅을 활용한 분류기 생성
    clf_bagging_svc = BaggingClassifier(base_estimator = SVC(), 
                                        n_estimators = i,               # n_estimator : 개별 학습기의 개수
                                        random_state = 0)               # 고정
    
    # 데이터 학습
    clf_bagging_svc.fit(X_tn_std, y_tn)

    # 데이터 예측
    pred_bagging_svc = clf_bagging_svc.predict(X_te_std) 
    
    # 데이터 정확도 평가
    accuracy = accuracy_score(y_te, pred_bagging_svc)
    
    # 분류 리포트 확인
    class_report = classification_report(y_te, pred_bagging_svc)
    
    # 모델 상황 확인
    
    print("모델 분류기 개수: ", i )
    print("모델 정확도: ", accuracy)
    print("모델 분류 리포트")
    print(class_report)
    print("========================================================")

모델 분류기 개수:  11
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

모델 분류기 개수:  12
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00         8

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

모델 분류기 개수:  13
모델 정확도:  1.0
모델 분류 리포트
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1   

### 결과

SVM 으로 실험한 결과 분류기의 개수가 크게 영향을 미치지 않으면서 성능은 거의 비슷하게 나왔다.

다만 성능이 떨어졌던 부분이 3개였던 부분인데, 정확히 왜 그런지는 내 머리로 이해할 수 없다...

---

배깅의 방식을 돌이켜보면, 여러 분류기의 분류 방식을 거쳐서, 개별 분류기들의 분류 결과를 종합하려 최종 분류기의 성능을 향상하는 방법이다.

분류 결과를 종합하는 방법이 같다면 분류기의 개수가 일정 수준이상 늘어나더라도 성능은 변하지 않는 선이 있으며, 

이때 분류기에 따라서 늘어나거나 줄어들 수 있다.